In [1]:
import tensorflow as tf
tf.test.is_built_with_cuda()

2022-08-03 09:38:51.704235: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


True

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-08-03 09:36:14.174627: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-03 09:36:14.191219: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2599735000 Hz
2022-08-03 09:36:14.194365: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cbed0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-03 09:36:14.194382: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-03 09:36:14.197934: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-03 09:36:14.621764: I tensorflow/compiler/xla/service/service.cc:168] XLA ser

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14905752582725223723,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1629047261043988019
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1102582180494838880
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22327264512
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11673741058215777109
 physical_device_desc: "device: 0, name: Tesla P40, pci bus id: 0000:0f:00.0, compute capability: 6.1"]

In [1]:
import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

2022-08-03 14:57:33.968127: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-08-03 14:57:35.290446: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-03 14:57:35.336644: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0e:00.0 name: Tesla P40 computeCapability: 6.1
coreClock: 1.531GHz coreCount: 30 deviceMemorySize: 22.38GiB deviceMemoryBandwidth: 323.21GiB/s
2022-08-03 14:57:35.336683: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-08-03 14:57:35.339011: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-08-03 14:57:35.341407: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2022

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
import pathlib
data_dir = pathlib.Path('./flower_photos/flower_photos')

In [6]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [5]:
batch_size = 32
img_height = 224
img_width = 224

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


2022-08-03 14:57:46.385199: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-03 14:57:46.394041: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2599735000 Hz
2022-08-03 14:57:46.397242: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4d0dba0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-03 14:57:46.397256: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-03 14:57:46.594095: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4d7a400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-08-03 14:57:46.594122: I tensorfl

In [7]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [8]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [11]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

2022-08-03 14:58:04.142728: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:172] Filling up shuffle buffer (this may take a while): 61 of 1000


0.0 1.0


2022-08-03 14:58:07.635074: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:221] Shuffle buffer filled.


In [12]:
num_classes = 5
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

In [13]:
model.compile(optimizer='adam',
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 64)        0

In [14]:
epochs = 10
history = model.fit(
train_ds,
validation_data=val_ds,
epochs = epochs
)

Epoch 1/10


2022-08-03 14:58:13.819461: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-08-03 14:58:14.108802: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


92/92 [==============================] - 6s 70ms/step - loss: 1.3980 - accuracy: 0.4166 - val_loss: 1.1094 - val_accuracy: 0.5504
Epoch 2/10
92/92 [==============================] - 3s 32ms/step - loss: 1.0270 - accuracy: 0.6035 - val_loss: 0.9750 - val_accuracy: 0.6240
Epoch 3/10
92/92 [==============================] - 2s 26ms/step - loss: 0.8058 - accuracy: 0.6962 - val_loss: 1.0084 - val_accuracy: 0.5913
Epoch 4/10
92/92 [==============================] - 2s 26ms/step - loss: 0.5544 - accuracy: 0.7987 - val_loss: 0.9868 - val_accuracy: 0.6567
Epoch 5/10
92/92 [==============================] - 2s 24ms/step - loss: 0.3064 - accuracy: 0.8968 - val_loss: 1.1884 - val_accuracy: 0.6431
Epoch 6/10
92/92 [==============================] - 2s 23ms/step - loss: 0.1538 - accuracy: 0.9523 - val_loss: 1.4350 - val_accuracy: 0.6335
Epoch 7/10
92/92 [==============================] - 2s 25ms/step - loss: 0.1011 - accuracy: 0.9710 - val_loss: 1.7185 - val_accuracy: 0.5845
Epoch 8/10
92/92 [======